<a href="https://colab.research.google.com/github/sungkwangsong/EasyOCR/blob/master/notebooks/hiBERT_%EC%97%B0%EA%B5%AC%EC%9E%AC%EB%8B%A8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
HBN_DATASETS_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/datasets/hibrainnet'
HBN_MODELS_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet'
HBN_EMBEDDINGS_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/embeddings/hibrainnet'
HBN_ARTICLES_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/hbn-data-lake/papers/kci/data'

In [3]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split

# 시드 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# KoSimCSE-BERT 모델 및 토크나이저 로드
model_name = "BM-K/KoSimCSE-bert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.to(device)

# 한국연구재단 분류 데이터 로드
def load_nrf_data(file_path, encoding='utf-8'):
    df = pd.read_csv(file_path, encoding=encoding)
    print(f"데이터 로드 완료. 형태: {df.shape}")
    print(f"컬럼: {df.columns.tolist()}")
    return df

# 데이터셋 클래스 정의 (SimCSE 방식)
class NRFDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # 각 텍스트를 두 번 토큰화 (dropout을 통한 데이터 증강)
        text = self.texts[idx]

        # 첫 번째 인코딩
        inputs1 = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # 두 번째 인코딩 (동일 텍스트, 다른 dropout 마스크)
        inputs2 = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids1': inputs1['input_ids'].squeeze(),
            'attention_mask1': inputs1['attention_mask'].squeeze(),
            'input_ids2': inputs2['input_ids'].squeeze(),
            'attention_mask2': inputs2['attention_mask'].squeeze(),
        }

# SimCSE 손실 함수 정의
class SimCSELoss(torch.nn.Module):
    def __init__(self, temperature=0.05):
        super(SimCSELoss, self).__init__()
        self.temperature = temperature
        self.cos_sim = torch.nn.CosineSimilarity(dim=-1)
        self.loss_fct = torch.nn.CrossEntropyLoss()

    def forward(self, embeddings_a, embeddings_b):
        batch_size = embeddings_a.size(0)

        # 코사인 유사도 계산
        cos_sim_matrix = self.cos_sim(embeddings_a.unsqueeze(1), embeddings_b.unsqueeze(0)) / self.temperature

        # 레이블: 대각선 요소(자기 자신)가 양성 쌍
        labels = torch.arange(batch_size, device=cos_sim_matrix.device)

        return self.loss_fct(cos_sim_matrix, labels)

# 학습 함수
def train(model, data_loader, optimizer, loss_fn, device, epoch):
    model.train()
    total_loss = 0
    valid_batches = 0

    progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}")

    for batch in progress_bar:
        try:
            optimizer.zero_grad()

            # 배치 데이터를 GPU로 이동
            input_ids1 = batch['input_ids1'].to(device)
            attention_mask1 = batch['attention_mask1'].to(device)
            input_ids2 = batch['input_ids2'].to(device)
            attention_mask2 = batch['attention_mask2'].to(device)

            # 배치가 비어있는지 확인
            if input_ids1.size(0) == 0 or input_ids2.size(0) == 0:
                continue

            # 첫 번째 문장 임베딩
            outputs1 = model(input_ids=input_ids1, attention_mask=attention_mask1)
            embeddings1 = outputs1.last_hidden_state[:, 0]  # [CLS] 토큰 임베딩

            # 두 번째 문장 임베딩
            outputs2 = model(input_ids=input_ids2, attention_mask=attention_mask2)
            embeddings2 = outputs2.last_hidden_state[:, 0]  # [CLS] 토큰 임베딩

            # 손실 계산
            loss = loss_fn(embeddings1, embeddings2)

            # 역전파
            loss.backward()

            # 그래디언트 클리핑 (폭발 방지)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += loss.item()
            valid_batches += 1
            progress_bar.set_postfix({'loss': total_loss / (valid_batches if valid_batches > 0 else 1)})

        except Exception as e:
            print(f"배치 처리 중 오류 발생: {e}")
            continue

    return total_loss / (valid_batches if valid_batches > 0 else 1)

# 평가 함수
def evaluate(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            # 배치 데이터를 GPU로 이동
            input_ids1 = batch['input_ids1'].to(device)
            attention_mask1 = batch['attention_mask1'].to(device)
            input_ids2 = batch['input_ids2'].to(device)
            attention_mask2 = batch['attention_mask2'].to(device)

            # 첫 번째 문장 임베딩
            outputs1 = model(input_ids=input_ids1, attention_mask=attention_mask1)
            embeddings1 = outputs1.last_hidden_state[:, 0]

            # 두 번째 문장 임베딩
            outputs2 = model(input_ids=input_ids2, attention_mask=attention_mask2)
            embeddings2 = outputs2.last_hidden_state[:, 0]

            # 손실 계산
            loss = loss_fn(embeddings1, embeddings2)
            total_loss += loss.item()

    return total_loss / len(data_loader)

# 메인 함수
def main():
    # 파일 경로 설정
    file_path = f"{HBN_DATASETS_DIR_PATH}/한국연구재단_전공분류 - 학습용(중분류포함).csv"

    # 데이터 로드
    df = load_nrf_data(file_path, encoding='utf-8')

    # 컬럼명 확인 및 실제 컬럼명으로 접근
    print("정확한 컬럼명:")
    for col in df.columns:
        print(f"- {col}")

    # 학습 데이터 준비
    # 키워드와 설명을 결합하여 텍스트 생성
    texts = []

    # 컬럼명 추출 - 실제 컬럼명과 일치하는지 확인
    keyword_col = [col for col in df.columns if 'KEYWORDS' in col][0]
    description_col = [col for col in df.columns if 'DESCRIPTION' in col][0]

    print(f"사용할 키워드 컬럼: {keyword_col}")
    print(f"사용할 설명 컬럼: {description_col}")

    for _, row in df.iterrows():
        keywords = str(row[keyword_col])
        description = str(row[description_col])

        # 처음 5개 행만 출력하여 데이터 확인
        if _ < 5:
            print(f"행 {_+1} 샘플:")
            print(f"  키워드: {keywords[:100]}...")
            print(f"  설명: {description[:100]}...")

        # 키워드와 설명이 모두 있는 경우만 학습 데이터에 추가
        if keywords != 'nan' and description != 'nan' and keywords.strip() and description.strip():
            # 텍스트 길이가 너무 짧지 않은 경우만 추가 (최소 10자)
            if len(keywords) > 10:
                texts.append(keywords)
            if len(description) > 10:
                texts.append(description)
            # 키워드와 설명을 결합한 텍스트도 추가
            combined = f"{keywords} {description}"
            if len(combined) > 20:  # 결합 텍스트는 더 길어야 함
                texts.append(combined)

    print(f"전처리된 텍스트 수: {len(texts)}")

    # 텍스트 데이터가 너무 적은 경우 경고
    if len(texts) < 100:
        print(f"경고: 학습 데이터가 매우 적습니다 ({len(texts)}개). 결과가 제한적일 수 있습니다.")

        # 데이터 증강: 텍스트 조각화를 통한 추가 데이터 생성
        augmented_texts = []
        for text in texts:
            if len(text) > 50:
                # 긴 텍스트는 여러 조각으로 나누어 추가 데이터 생성
                chunks = [text[i:i+50] for i in range(0, len(text), 25)]  # 50자씩, 25자 겹침
                augmented_texts.extend(chunks)

        texts.extend(augmented_texts)
        print(f"데이터 증강 후 텍스트 수: {len(texts)}")

    # 학습/검증 데이터 분할
    train_texts, val_texts = train_test_split(texts, test_size=0.1, random_state=42)

    print(f"학습 데이터 크기: {len(train_texts)}")
    print(f"검증 데이터 크기: {len(val_texts)}")

    # 데이터셋 및 데이터로더 생성
    train_dataset = NRFDataset(train_texts, tokenizer)
    val_dataset = NRFDataset(val_texts, tokenizer)

    # 배치 크기 조정 (데이터셋 크기에 따라)
    batch_size = min(16, max(4, len(train_texts) // 10))  # 데이터셋의 1/10, 최소 4, 최대 16
    print(f"배치 크기: {batch_size}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # 손실 함수 및 옵티마이저 설정
    loss_fn = SimCSELoss(temperature=0.05)
    optimizer = AdamW(model.parameters(), lr=2e-5)

    # 학습 실행
    num_epochs = 5
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # 학습
        train_loss = train(model, train_loader, optimizer, loss_fn, device, epoch)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")

        # 평가
        val_loss = evaluate(model, val_loader, loss_fn, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        # 최고 성능 모델 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print(f"Saving the best model with validation loss: {val_loss:.4f}")
            model_save_path = f"{HBN_MODELS_DIR_PATH}/hiBERT"
            model.save_pretrained(model_save_path)
            tokenizer.save_pretrained(model_save_path)

    print("학습 완료!")

    # 간단한 테스트
    test_sentence = "인공지능 딥러닝 연구"
    print(f"테스트 문장: '{test_sentence}'")

    # 파인튜닝된 모델로 임베딩 생성
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(test_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0].cpu().numpy()

        print(f"임베딩 형태: {embedding.shape}")
        print("임베딩 벡터 (처음 5개 값):", embedding[0][:5])

# 스크립트 실행
if __name__ == "__main__":
    main()

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Epoch 4/5, Validation Loss: 0.0000
Saving the best model with validation loss: 0.0000


Epoch 5: 100%|██████████| 258/258 [00:48<00:00,  5.36it/s, loss=0.000496]


Epoch 5/5, Train Loss: 0.0005


Evaluating: 100%|██████████| 29/29 [00:02<00:00, 12.86it/s]

Epoch 5/5, Validation Loss: 0.0000
학습 완료!
테스트 문장: '인공지능 딥러닝 연구'
임베딩 형태: (1, 768)
임베딩 벡터 (처음 5개 값): [ 1.0093793  -1.1689878   0.63222384 -0.16977587  0.8310323 ]


In [6]:
import os
import json
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import glob

class PaperSearchSystem:
    def __init__(self, model_path, device=None):
        """
        논문 검색 시스템 초기화

        Args:
            model_path: 파인튜닝된 hiBERT 모델 경로
            device: 연산 장치 (None이면 자동 감지)
        """
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        print(f"Using device: {self.device}")

        # 모델 및 토크나이저 로드
        try:
            # 파인튜닝된 모델 로드
            self.tokenizer = BertTokenizer.from_pretrained(model_path)
            self.model = BertModel.from_pretrained(model_path)
            print(f"파인튜닝된 hiBERT 모델을 성공적으로 로드했습니다: {model_path}")
        except Exception as e:
            print(f"모델 로드 중 오류 발생: {e}")
            print(f"기본 KoSimCSE-BERT 모델을 대신 로드합니다.")
            self.tokenizer = BertTokenizer.from_pretrained("BM-K/KoSimCSE-bert")
            self.model = BertModel.from_pretrained("BM-K/KoSimCSE-bert")

        self.model.to(self.device)
        self.model.eval()

        # 논문 데이터 및 임베딩 저장소
        self.papers = []
        self.paper_embeddings = None

    def load_papers_from_dir(self, dir_path):
        """
        디렉토리에서 JSON 논문 파일들을 로드

        Args:
            dir_path: 논문 JSON 파일이 있는 디렉토리 경로
        """
        json_files = glob.glob(os.path.join(dir_path, "*.json"))
        print(f"Found {len(json_files)} JSON files in directory: {dir_path}")

        for file_path in tqdm(json_files, desc="Loading papers"):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    paper_data = json.load(f)

                # 기본 값 설정 (파일 구조에 따라 다를 수 있음)
                paper_info = {
                    'title': '',
                    'abstract': '',
                    'keywords': [],
                    'authors': [],
                    'category': '',
                    'file_path': file_path
                }

                # 제목 추출
                article_info = paper_data.get('articleInfo', {})
                title_group = article_info.get('title-group', {})

                if isinstance(title_group.get('article-title'), list):
                    for title_item in title_group.get('article-title', []):
                        if title_item.get('@lang') == 'original':
                            paper_info['title'] = title_item.get('#text', '')
                            break
                elif isinstance(title_group.get('article-title'), dict):
                    paper_info['title'] = title_group.get('article-title', {}).get('#text', '')

                # 초록 추출
                abstract_group = article_info.get('abstract-group', {})
                if isinstance(abstract_group.get('abstract'), list):
                    for abstract_item in abstract_group.get('abstract', []):
                        if abstract_item.get('@lang') == 'original':
                            paper_info['abstract'] = abstract_item.get('#text', '')
                            break
                elif isinstance(abstract_group.get('abstract'), dict):
                    paper_info['abstract'] = abstract_group.get('abstract', {}).get('#text', '')

                # 키워드 추출
                detail_info = paper_data.get('detailInfo', {})
                detail_article_info = detail_info.get('articleInfo', {})
                keyword_group = detail_article_info.get('keyword-group', {})

                if isinstance(keyword_group.get('keyword'), list):
                    paper_info['keywords'] = keyword_group.get('keyword', [])
                elif isinstance(keyword_group.get('keyword'), str):
                    paper_info['keywords'] = [keyword_group.get('keyword', '')]

                # 저자 추출
                author_group = article_info.get('author-group', {})
                if isinstance(author_group.get('author'), list):
                    for author in author_group.get('author', []):
                        if isinstance(author, str):
                            paper_info['authors'].append(author)
                        elif isinstance(author, dict):
                            author_name = author.get('#text', '')
                            if author_name:
                                paper_info['authors'].append(author_name)
                elif isinstance(author_group.get('author'), dict):
                    author_name = author_group.get('author', {}).get('#text', '')
                    if author_name:
                        paper_info['authors'].append(author_name)

                # 카테고리 추출
                paper_info['category'] = article_info.get('article-categories', '')

                # 논문 정보가 제대로 존재하는 경우만 추가
                if paper_info['title'] or paper_info['abstract']:
                    self.papers.append(paper_info)

            except Exception as e:
                print(f"Error loading paper from {file_path}: {e}")

        print(f"Successfully loaded {len(self.papers)} papers.")

    def create_paper_embeddings(self, batch_size=4):
        """
        논문의 임베딩 벡터 생성

        Args:
            batch_size: 배치 크기
        """
        if not self.papers:
            raise ValueError("Please load papers first using load_papers_from_dir()")

        # 제목, 초록, 키워드 결합
        texts = []
        for paper in self.papers:
            title = paper.get('title', '')
            abstract = paper.get('abstract', '')
            keywords = ' '.join(paper.get('keywords', []))

            # 텍스트 결합 (제목이 중요하므로 2번 포함)
            text = f"{title} {title} {abstract} {keywords}"
            texts.append(text)

        # 배치 처리로 임베딩 생성
        embeddings = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Creating paper embeddings"):
            batch_texts = texts[i:i+batch_size]

            # 토큰화 및 텐서 생성
            inputs = self.tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            )

            # GPU로 이동
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # 임베딩 생성
            with torch.no_grad():
                outputs = self.model(**inputs)
                batch_embeddings = outputs.last_hidden_state[:, 0].cpu().numpy()
                embeddings.extend(batch_embeddings)

        self.paper_embeddings = np.array(embeddings)
        print(f"Created embeddings with shape: {self.paper_embeddings.shape}")

    def save_embeddings(self, output_file):
        """
        생성된 임베딩 저장

        Args:
            output_file: 저장할 파일 경로
        """
        if self.paper_embeddings is None:
            raise ValueError("No embeddings to save. Create embeddings first.")

        # 논문 정보와 임베딩을 함께 저장
        data_to_save = {
            'papers': self.papers,
            'embeddings': self.paper_embeddings.tolist()
        }

        # 디렉토리 생성
        os.makedirs(os.path.dirname(os.path.abspath(output_file)), exist_ok=True)

        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(data_to_save, f, ensure_ascii=False, indent=2)

        print(f"Saved papers and embeddings to {output_file}")

    def load_embeddings(self, embeddings_file):
        """
        저장된 임베딩과 논문 정보 로드

        Args:
            embeddings_file: 임베딩 파일 경로
        """
        with open(embeddings_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        self.papers = data.get('papers', [])
        self.paper_embeddings = np.array(data.get('embeddings', []))

        print(f"Loaded {len(self.papers)} papers and embeddings with shape: {self.paper_embeddings.shape}")

    def search(self, query, top_k=5, min_similarity=0.3):
        """
        쿼리와 의미적으로 유사한 논문 검색

        Args:
            query: 검색 쿼리
            top_k: 반환할 상위 결과 수
            min_similarity: 최소 유사도 점수

        Returns:
            검색 결과 (논문 정보와 유사도 점수)
        """
        if not self.papers or self.paper_embeddings is None:
            raise ValueError("Papers and embeddings must be loaded before searching")

        # 쿼리 임베딩 생성
        inputs = self.tokenizer(
            query,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)
            query_embedding = outputs.last_hidden_state[:, 0].cpu().numpy()

        # 코사인 유사도 계산
        similarities = cosine_similarity(query_embedding, self.paper_embeddings)[0]

        # 유사도 점수에 따라 정렬된 인덱스 가져오기
        sorted_indices = similarities.argsort()[::-1]

        # 결과 생성
        results = []
        for idx in sorted_indices:
            score = similarities[idx]
            if score < min_similarity or len(results) >= top_k:
                break

            paper_info = self.papers[idx].copy()
            paper_info['similarity_score'] = float(score)
            results.append(paper_info)

        return results

def display_search_results(results):
    """
    검색 결과를 화면에 출력

    Args:
        results: 검색 결과 리스트
    """
    if not results:
        print("검색 결과가 없습니다.")
        return

    print(f"\n검색 결과 ({len(results)}개):")
    print("=" * 80)

    for i, paper in enumerate(results):
        print(f"{i+1}. [{paper['similarity_score']:.4f}] {paper['title']}")
        if paper.get('authors'):
            print(f"   저자: {', '.join(paper['authors'])}")
        if paper.get('category'):
            print(f"   분야: {paper['category']}")
        if paper.get('keywords'):
            print(f"   키워드: {', '.join(paper['keywords'])}")
        if paper.get('abstract'):
            # 초록 앞부분만 표시
            abstract = paper['abstract']
            if len(abstract) > 200:
                abstract = abstract[:200] + "..."
            print(f"   초록: {abstract}")
        print(f"   파일: {os.path.basename(paper['file_path'])}")
        print("-" * 80)

def main():
    # 설정 변수들
    model_path = f"{HBN_MODELS_DIR_PATH}/hiBERT"  # 모델 경로
    papers_dir = HBN_ARTICLES_DIR_PATH       # 논문 디렉토리 경로
    embeddings_file = f"{HBN_EMBEDDINGS_DIR_PATH}/articles_embeddings.json"  # 임베딩 파일 경로
    create_embeddings = False                  # 임베딩 새로 생성 여부
    top_k = 5                                  # 검색 결과 개수

    # 테스트용 검색 쿼리들
    test_queries = [
        "온톨로지 기반 검색",
        "인공지능 알고리즘",
        "데이터 마이닝",
        "추론 시스템",
        "지식 그래프"
    ]

    # 검색 시스템 초기화
    search_system = PaperSearchSystem(model_path)

    # 임베딩 생성 또는 로드
    if create_embeddings:
        # 논문 데이터 로드
        search_system.load_papers_from_dir(papers_dir)
        # 임베딩 생성
        search_system.create_paper_embeddings()
        # 임베딩 저장
        search_system.save_embeddings(embeddings_file)
    elif os.path.exists(embeddings_file):
        # 저장된 임베딩 로드
        search_system.load_embeddings(embeddings_file)
    else:
        # 논문 데이터 로드
        search_system.load_papers_from_dir(papers_dir)
        # 임베딩 생성
        search_system.create_paper_embeddings()
        # 임베딩 저장
        search_system.save_embeddings(embeddings_file)

    # 각 테스트 쿼리로 검색 수행
    for query in test_queries:
        print(f"\n[검색어: '{query}']")
        try:
            results = search_system.search(query, top_k=top_k)
            display_search_results(results)
        except Exception as e:
            print(f"검색 중 오류 발생: {e}")

    # 대화형 모드
    print("\n===== hiBERT 한국 논문 검색 시스템 =====")
    print("'quit' 또는 'exit'를 입력하면 종료합니다.\n")

    while True:
        query = input("\n검색어를 입력하세요: ")
        if query.lower() in ['quit', 'exit', '종료']:
            print("검색 시스템을 종료합니다.")
            break

        try:
            results = search_system.search(query, top_k=top_k)
            display_search_results(results)
        except Exception as e:
            print(f"검색 중 오류가 발생했습니다: {e}")

if __name__ == "__main__":
    main()

Using device: cuda
파인튜닝된 hiBERT 모델을 성공적으로 로드했습니다: /content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet/hiBERT
Loaded 698 papers and embeddings with shape: (698, 768)

[검색어: '온톨로지 기반 검색']

검색 결과 (1개):
1. [0.4094] 온톨로지 기반에서 연관 마이닝 방법을 이용한 지식 추론 알고리즘 연구
   저자: 황현숙(부경대학교), 이준연(동명대학교)
   분야: 전자/정보통신공학
   키워드: Ontology(온톨로지), Association Mining(연관마이닝), Inference Algorithm(추론알고리즘), User Profile Modeling(사용자 프로파일 모델링), Databases(데이터베이스)
   초록: 정보 검색에 대한 연구는 방대한 데이터에서 원하는 검색 정보를 제공할 뿐 만 아니라 개인의 취향에 따른 맞춤 검색 및 추론된 지식을 제공하는 데 초점을 두고 있다. 본 논문의 목적은 데이터를 개념화하여 분류 및 정의할 수 있는 온톨로지 구조를 기반으로 숨어있는 지식을 발견하여 개인 맞춤 검색을 제공하는 추론 알고리즘에 대해 연구하는 것이다. 현재의 검색에서...
   파일: KCI-ART001293627-온톨로지 기반에서 연관 마이닝 방법을 이용한 지식 추론 알고리즘 연구.json
--------------------------------------------------------------------------------

[검색어: '인공지능 알고리즘']

검색 결과 (2개):
1. [0.3407] 인공지능을 활용하는 행정작용에 대한 법적 고찰
   저자: 강현호(성균관대학교)
   분야: 법학
   키워드: 인공지능(Künstliche Intelligenz), 설명요구권(Recht au